# Détection et mise en correspondance de primitives


## 1. Déscription

Le but de la détection et de la mise en correspondance de primitives est d'établir le lien entre un point dans une image *X* et son correspondant dans une autre image *Y*. Ces correspondances peuvent ensuite être utilisées pour assembler plusieurs images dans un panorama.

Dans ce devoir, vous allez écrire du code pour détecter les primitives discriminantes (qui sont raisonnablement invariantes à la translation, la rotation et le changement dans l'éclairage) dans une image et trouver les meilleures primitives correspondantes dans une autre image. Le devoir comprend trois parties : détection des primitives, description des primitives et mise en correspondance des primitives. 

Un certain nombre de classes et de méthodes vous sont déjà fournis. Il est primordial de comprendre le rôle de chaque classe et fonction avant d'essayer de modifier le code. Prenez donc le temps de consulter les fichiers sources et les commentaires fournis. 

**<span style='color:Red'> Le code que vous devez écrire et toutes les modifications requises seront dans le fichier `features.py`</span>.**

Pour vous aider à visualiser vos résultats et déboguer votre implémentation, une interface graphique est fournit (fichier **`featuresUI.py`**). Cette interface utilisateur permet d'afficher les primitives détectées et les meilleures correspondances entre deux image. 

Un exemple de détecteur/descripteur de primitive appelé "[Oriented FAST and Rotated BRIEF (ORB)](https://medium.com/analytics-vidhya/introduction-to-orb-oriented-fast-and-rotated-brief-4220e8ec40cf)" est également fournit pour comparaison. Cette technique est très populaire en Vision par Ordinateur et se propose comme alternative aux algorithmes [SIFT](https://docs.opencv.org/3.4.9/da/df5/tutorial_py_sift_intro.html) et [SURF](https://docs.opencv.org/3.4.9/df/dd2/tutorial_py_surf_intro.html).


In [ ]:
'''
Les cellules de code dans ce blocs-note vous permettent de tester votre 
implémentation des différentes tâches de ce devoir. Si vos tests échouent, 
vous devez vérifier pourquoi ils ont échoué. En particulier, faites attention 
aux tolérances/seuils utilisés dans ce bloc-notes. Il est possible que votre 
réponse soit correcte, mais elle tombe à peine en dehors de la plage de tolérance.
'''
# Numpy est le paquetage principal utilisé pour le calcul scientifique dans Python. 
import numpy as np

# cv2 est le paquetage OpenCV pour Python - il est utilisé pour implémenter/utiliser 
# des fonctionalités en traintement d'image et/ou en vision par ordinateur
import cv2

# Ce module fournit une interface standard pour extraire, formater et imprimer 
# les traces de pile des programmes Python
import traceback

# PIL (Python Image Library) est un paquetage utilisé pour manipuler les images sous Python 
from PIL import Image

# Quelques instructions supplémentaires pour que le notebook recharge les modules externes en python;
# voir http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Chargement du fichier de l'étudiant - c.-à-d. le fichier features.py
import features

# Sauvegarde et chargement de points cv2
def pickle_cv2(arr):
    index = []
    for point in arr:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, point.class_id)
        index.append(temp)
    return np.array(index, dtype=object)

def unpickle_cv2(arr):
    index = []
    for point in arr:
        temp = cv2.KeyPoint(x=point[0][0],
                            y=point[0][1],
                            _size=point[1], 
                            _angle=point[2], 
                            _response=point[3], 
                            _octave=point[4], 
                            _class_id=point[5])
        index.append(temp)
    return np.array(index)

# Fonctions pour tester l'exactitude de deux tableaux élément-par-élément
def compare_array(arr1, arr2):    
    return np.allclose(arr1,arr2,rtol=1e-3,atol=1e-5)

# Fonction pour comparer deux points cv2
def compare_cv2_points(pnt1, pnt2):
    if not np.isclose(pnt1.pt[0],pnt2.pt[0],rtol=1e-3,atol=1e-5): return False
    if not np.isclose(pnt1.pt[1],pnt2.pt[1],rtol=1e-3,atol=1e-5): return False
    if not np.isclose(pnt1.angle,pnt2.angle,rtol=1e-3,atol=1e-5): return False
    if not np.isclose(pnt1.response,pnt2.response,rtol=1e-3,atol=1e-5): return False
    return True

# fonction appelée pour tester les différentes tâches à implémenter
def try_this(todo, run, truth, compare, *args, **kargs):
    '''
    Exécute une fonction, teste le résultat avec 'compare', et affiche un message d'erreur 
    si ça ne fonctionne pas
    @arg todo (int or str): Le numéro du TODO 
    @arg run (func): La fonction à exécuter
    @arg truth (any): Le résultat correct  de la fonction testée
    @arg compare (func->bool): retourne un booléen de la comparaison de la sortie de la fonction `run` avec le résultat correct
    @arg *args (any): Arguments supplémentaires éventuels à passer à la fonction `run`
    @arg **kargs (any): Arguments supplémentaires éventuels à passer à la fonction `compare`

    @return (int): Le nombre de tests qui ont échoué
    '''
    print('Starting test for TODO {}'.format(todo))
    failed = 0
    try:
        output = run(*args)
        print("ok")
    except Exception as e:
        traceback.print_exc()
        print("TODO {} threw an exception, see exception below\n".format(todo))
        return
    if type(output) is list or type(output) is tuple:
        for i in range(len(output)):
            if not compare(output[i], truth[i], **kargs):
                print("TODO {} doesn't pass test: {}".format(todo, i))
                failed+=1
    else:
        if not compare(output, truth, **kargs):
            print("TODO {} doesn't pass test".format(todo))
            failed+=1
    return failed


image = np.array(Image.open('resources/triangle1.jpg'))
grayImage = cv2.cvtColor(image.astype(np.float32)/255.0, cv2.COLOR_BGR2GRAY)

'''
Chargement des tableaux numpy contenant les résultats de triangle1.jpg.

Ces tableaux sont accessibles à l'aide de loaded['<lettre>']. 
Par exemple, la réponse correcte pour le test 2 est 'c', donc pour voir 
la réponse correcte pour le test 2 vous pouvez inspecter loaded['c'].
Remarque importante : NumPy n'affiche pas l'ensemble du tableau s'il est 
très grand --- vous devez afficher des parties plus petites (e.g., 
print( repr(loaded['c'][0]) ) ).
'''
loaded = np.load('resources/arrays.npz', allow_pickle=True)
d = unpickle_cv2(loaded['d_proc'])


## 2. Tâches principales

### 2.1. Détection de points-clés ( 35 points )

Dans cette étape, vous identifierez les points-clés dans l'image à l'aide de la méthode de détection des coins Harris. Pour chaque point de l'image, vous devez réaliser les étapes suivantes (voir les slides du cours pour plus de détails) : 

1. Définissez une fenêtre de pixels autour de ce point.


2. Calculez la matrice de Harris *H* pour (la fenêtre autour de) ce point, définie comme
 
    $
     \begin{align}
     H & = \sum_p w_p \nabla I_p (\nabla I_p)^T
     \\
     & = \sum_p w_p \begin{bmatrix} 
                 {I_{x_p}}^2 & I_{x_p} I_{y_p} \\ 
                  I_{x_p} I_{y_p} & {I_{y_p}}^2
               \end{bmatrix}               
     \\
     & = \sum_p     \begin{bmatrix} 
                 w_p {I_{x_p}}^2 & w_p I_{x_p} I_{y_p} \\ 
                 w_p I_{x_p} I_{y_p} & w_p  {I_{y_p}}^2
               \end{bmatrix}               
     \\
     & = \begin{bmatrix} 
       \sum_p w_p {I_{x_p}}^2 & \sum_p w_p I_{x_p} I_{y_p} \\ 
       \sum_p w_p I_{x_p} I_{y_p} & \sum_p w_p  {I_{y_p}}^2
     \end{bmatrix}    
     \end{align}    
    $

    où la somme est sur tous les pixels *p* de la fenêtre. 
    
    *H* est une matrice $2 \times 2$. $I_{x_p}$ est la dérivée *x* de l'image au point *p*, la notation est similaire pour la dérivée *y*. Vous devez utiliser l'opérateur de Sobel $3 \times 3$ pour calculer les dérivées *x* et *y* (utilisez le mode "réflexion" pour extrapoler les pixels aux bords de l'image). Les poids $w_p$ doivent être circulairement symétriques (pour l'invariance à la rotation) - utilisez un masque gaussien $5 \times 5$ avec $\sigma = 0.5$.<br>


3. Utilisez ensuite *H* pour calculer la fonction d'intensité de coin $c(H)$ à chaque pixel. En ce sens, utilisez la définition alternative ci-dessous du score $c(H)$ où *k* est une constante déterminée empiriquement; utilisez la valeur $k = 0.1$.
 
    $    
     c(H) = \lambda_1 \lambda_2 - k \cdot (\lambda_1 + \lambda_2)^2  = det(H) - k \cdot (trace(H))^2     
    $


4. Vous aurez également besoin de l'angle d'orientation à chaque pixel. Utilisez l'angle du gradient comme une approximation. L'angle zéro pointe vers la droite et les angles positifs sont dans le sens de rotation des aiguilles d'une montre. **Remarque : Ne pas calculer l'orientation par la méthode des vecteurs propres**.


5. Sélectionnez les points-clés les plus forts et qui correspondent aux maxima locaux de $c(H)$. Utilisez une fenêtre $7 \times 7$ pour vos calculs.


####  Code à écrire

La fonction **`detectKeypoints`** dans **` HarrisKeypointDetector`** est l'une des principales fonctions que vous devez compléter avec les fonctions auxilières **`computeHarrisValues`** (calcule le score et l'orientation de Harris pour chaque pixel de l'image) et **`computeLocalMaxima`** (calcule un tableau de booléens qui indique pour chaque pixel s'il est un maximum local ou non). L'ensemble de ces fonctions implémente le détecteur de coins de Harris. 

Les fonctions suivantes vous seront peut-être utiles pour implémenter cette tâche :

- `scipy.ndimage.sobel`: Filtre l'image en entrée avec le filtre Sobel.
- `scipy.ndimage.gaussian _filter`: Filtre l'image en entrée avec un filtre gaussien.
- `scipy.ndimage.filters.maximum_ filter`: Filtre l'image en entrée avec un filtre maximum.
- `scipy.ndimage.filters.convolve`: Filtre l'image en entrée avec un noyau spécifique.

Vous pouvez consulter la documentation sur ces fonctions [ici](http://docs.scipy.org/doc/scipy/reference/ndimage.html).


In [ ]:
'''
Nous allons tester ici votre implémentation des TODOs 1, 2 et 3 dans features.py
sur l'exemple d'image triangles1.jpg - N'oubliez pas d'éxecuter la cellule précédente
de code Python pour initialiser l'environement de test.
'''

# construction de l'objet HKD pour la détection des points-clés
HKD = features.HarrisKeypointDetector()
# appel de la fonction de test pour le TO-DO n°1
try_this(1, HKD.computeHarrisValues, [loaded['a'],loaded['b']], compare_array, grayImage)

# patcher HKD afin que les futurs tests n'échouent pas si le dernier test a échoué
class HKD2(features.HarrisKeypointDetector):
  def computeHarrisValues(self,image):
    return loaded['a'],loaded['b']
HKD=HKD2()
# appel de la fonction de test pour le TO-DO n°2
try_this(2, HKD.computeLocalMaxima, loaded['c'], compare_array, loaded['a'])

# patcher HKD afin que les futurs tests n'échouent pas si le dernier test a échoué
class HKD3(HKD2):
  def computeLocalMaxima(self,image):
    return loaded['c']
HKD=HKD3()
# appel de la fonction de test pour le TO-DO n°3
try_this(3, HKD.detectKeypoints, d, compare_cv2_points, image)


### 2.2. Descripteur de primitive  ( 35 points )

Une fois les points d'intérêt identifiés, l'étape suivante consiste à trouver un *descripteur* pour chaque point-clé détecté. Ce descripteur sera la représentation que vous utiliserez pour comparer des primitives dans différentes images et voir si elles correspondent. Vous implémenterez deux descripteurs pour ce devoir : 

- En premier, vous implémenterez un descripteur simple conçu des valeurs d'intensité de pixel dans un voisinage $5 \times 5$ du point-clé. Cela devrait être facile à mettre en oeuvre et devrait fonctionner correctement quand les images comparées sont liées par une simple translation.


- Deuxièmement, vous implémenterez une version simplifiée du descripteur MOPS. En ce sens, implémentez la matrice qui vous permettra de transformer une fenêtre $40 \times 40$ en un patch de $8 \times 8$ pixels après l'avoir pivotée d'un angle $\theta$ autour du point-clé. La nouvelle orientation du point-clé devra être vers la droite. 

  Vous devez également normaliser les intensités du patch pour avoir une moyenne nulle et une variance égale à 1. Si la variance du patch avant normalisation est très proche de zéro (inférieure à $10^{-10}$ en valeur absolue), renvoyez simplement un descripteur entièrement nul pour éviter une erreur de division par zéro.
  
  Utilisez `cv2.warpAffine` de OpenCV pour effectuer la transformation. La fonction `warpAffine` requiert une matrice $2 \times 3$ en entrée. La façon la plus simple de générer cette matrice est de combiner plusieurs transformations basiques. Une séquence de translation $\mathbf{T_1}$, rotation $\mathbf{R}$, redimensionnement $\mathbf{S}$ et translation $\mathbf{T_2}$ produira la matrice escompté. Notez que les transformations sont combinées de droite à gauche, de sorte que la matrice de transformation est le produit matriciel $\mathbf{T_2\;S\;R\;T_1}$. Les figures ci-dessous illustrent la séquence.
  

![](warpAffine.png)


#### Code à écrire

Vous devez implémenter deux descripteurs de primitives dans les classes **`SimpleFeatureDescriptor`** et **` MOPSFeaturesDescriptor`**. La méthode **`describeFeatures`** de ces classes prend en entrée les informations de position et d'orientation des points-clés déjà calculés (par exemple par le détecteur Harris) et calcule les descripteurs pour ces points-clés. Ces descripteurs sont ensuite stockés dans un tableau Numpy bidimensionnel. Le nombre de lignes de ce tableau est égale au nombre de point-clés détectés. Le nombre de colonnes du tableau est égale à la dimension du descripteur retourné (par exemple, 25 pour le descripteur simple de primitives $5 \times 5$).

Pour l'implémentation de MOPS, vous devez créer une matrice pour transformer une fenêtre $40 \times 40$ en un patch de $8 \times 8$ pixels après l'avoir pivotée d'un angle $\theta$ autour du point-clé (comme décrit ci-dessus). 

Il est recommandé de consulter la [documentation opencv](https://docs.opencv.org/3.4.9/d4/d61/tutorial_warp_affine.html) sur **`cv2.warpAffine`**.

In [ ]:
'''
Nous allons tester ici votre implémentation des TODOs 4, 5 et 6 dans features.py
sur l'exemple d'image triangles1.jpg
'''

# construction de l'objet SFD pour un descripteur simple d'intensités
SFD = features.SimpleFeatureDescriptor()

# appel de la fonction de test pour le TO-DO n°4
try_this(4, SFD.describeFeatures, loaded['e'], compare_array, image, d)

# construction de l'objet MFD pour un descripteur MOPS
MFD = features.MOPSFeatureDescriptor()

# appel de la fonction de test pour le TO-DO n°5 et/ou n°6
try_this('5 and/or 6', MFD.describeFeatures, loaded['f'], compare_array, image, d)

### 2.3. Mise en correspondance de primitives ( 30 points )

Une fois les points-clés détectés et leurs descripteurs calculés pour un ensemble d'images, l'étape suivante consiste à établir les correspondances entre les différents points-clés identifiés dans les images (c.-à-d., étant donné une primitive dans une image *X*, trouver la meilleure primitive correspondante dans une autre image *Y*).


L'approche la plus simple est la suivante : Pour chaque descripteur d'une image X, le comparer aux descripteurs d'une image *Y* en calculant une *distance* scalaire. La meilleure correspondance est la paire de primitives qui produit la plus petite distance. Vous implémenterez deux fonctions de distance :

1. Somme des moindres carrés (SMC): il s'agit de la distance euclidienne au carré entre les deux descripteurs.


2. Ratio de distances : le rapport des distances SMC associées aux deux meilleurs descripteurs identifiés (c.-à-d. le ratio est égale à la distance SMC du descripteur le plus proche divisée par la distance SMC du second descripteur le plus proche).

#### Code à écrire

Dans cette tache, vous allez implémenter une fonction pour la mise en correspondance de primitives. Vous implémenterez la fonction **`matchFeatures`** de **` SSDFeatureMatcher`** et de **`RatioFeatureMatcher`**. Ces fonctions renvoient une liste d'objets [cv2.DMatch](https://docs.opencv.org/3.4.9/d4/de0/classcv_1_1DMatch.html). 

Vous devez initialiser l'attribut `queryIdx` de `cv2.DMatch` à l'index de la primitive dans la première image, l'attribut` trainIdx` à l'index de la primitive dans la deuxième image et l'attribut `distance` à la distance (ou le ratio, selon le cas) entre les deux primitives. 

Les fonctions `scipy.spatial.distance.cdist` et` numpy.argmin` pourrait vous être utiles lors de l'implémentation de cette tache.


In [ ]:
'''
En vous inspirant des tests précédents, vous pouvez implémenter ici vos tests
pour les TODOs 7 et 8 dans features.py
'''

SSDFM = features.SSDFeatureMatcher()

# TODO 7: tester la fonction de mise en correspondance 'matchFeatures' de la classe 'SSDFeatureMatcher' 
# TODO 8: tester la fonction de mise en correspondance 'matchFeatures' de la classe 'RatioFeatureMatcher'


## 3. Tâche bonus (20 points)

Des points supplémentaires sont accordés pour les solutions qui améliorent, d'au moins 15%, l'**A**ire **S**ous la **C**ourbe (ASC) moyenne pour le test utilisant le "ratio de distances". Une amélioration de 15% est interprétée comme une réduction de 15% de la valeur $(1 - ASC)$. C'est-à-dire la zone au-dessus de la courbe. Voici une suggestion (et nous vous encourageons également à proposer vos propres idées!)

 - Implémentez la méthode **`selectKeypointsANMS`** de la classe **`KeypointDetector`**. Cette méthode devrait réaliser la technique de suppression non maximale adaptative discutée dans l'[article MOPS](https://drive.google.com/open?id=1vly0rexasm-kM_lWEulJ-PiBfCDm8cdO)
  
Vous pouvez mesurer votre ASC moyenne en exécutant le benchmark de l'interface utilisateur **`featuresUI.py`** sur les cinq ensembles de données (bikes, graf, leuven, wall, yosemite). Nous évaluerons spécifiquement les performances de votre implémentation contre l'ensemble de données 'yosemite'. Les implémentations qui n'améliorent pas l'ASC moyenne d'au moins 15% ne seront pas notées.

Le bonus sera accordé en fonction du mérite de votre amélioration et de **sa justification**. Les modifications hyperparamétriques simples recevront moins de points supplémentaires que les améliorations significatives de l'algorithme.

Décrivez votre implémentation dans un fichier nommé **`Lisez-moi_adaptative.txt`** qui décrit vos modifications. 


## 4. Visualisation

L'interface utilisateur dans **`featuresUI.py`** est un outil pour vous aider à visualiser les détections de points-clés et les résultats de correspondance des primitives. Les fonctions de détection de points-clés et les méthodes de mise en correspondance sont appelées à partir de cette interface. N'hésitez pas à étendre les fonctionnalités de cette interface, mais n'oubliez pas que seul le code dans **`features.py`** sera noté.

En exécutant **`featuresUI.py`**, vous verrez une interface utilisateur dans laquelle vous avez les choix suivants :

- **Détection des points-clés** : <br>
  Vous pouvez charger une image et calculer les points d'intérêt avec leurs orientations.


- **Mise en correspondance des primitives** : <br>
  Ici, vous pouvez charger deux images et afficher les meilleures correspondances calculées à l'aide des algorithmes spécifiés.


- **Analyse** : <br>
  Après avoir spécifié le chemin d'accès au répertoire contenant l'ensemble de données, le programme exécutera les algorithmes spécifiés sur toutes les images et calculera les courbes ROC pour chaque image.


Egalement founit est un ensemble d'images de référence pour tester les performances de votre algorithme en fonction de différents types de variation contrôlée (c'est-à-dire, rotation, redimensionnement, éclairage, perspective, flou). Pour chacune de ces images, nous connaissons la transformation correcte et pouvons donc mesurer la précision de chacune de vos détection et/ou mise en correspondance de primitives. Gardez à l'esprit que votre code sera évalué numériquement, pas visuellement.

Vous devriez également sortir et prendre vos propres photos pour voir à quel point votre approche fonctionne sur des ensembles de données plus intéressants.


## 5. Quoi remettre

### 5.1 Règles de codage à respecter

- Vous pouvez utiliser les fonctions NumPy, SciPy et OpenCV2 pour implémenter des opérations mathématiques, de filtrage et de transformation. N'utilisez pas de fonctions qui implémentent la détection de points-clés ou la mise en correspondance de primitives - ça sera un zéro pour tout le devoir si vous le faites !


- Lorsque vous utilisez l'opérateur de Sobel ou le filtre gaussien, vous devez utiliser le mode « réflexion » pour avoir un gradient nul sur les bords.


- Voici une liste de fonctions potentiellement utiles (vous n'êtes pas obligé de les utiliser) :

      - scipy.ndimage.sobel
      - scipy.ndimage.gaussian_filter
      - scipy.ndimage.filters.convolve
      - scipy.ndimage.filters.maximum_filter
      - scipy.spatial.distance.cdist
      - cv2.warpAffine
      - np.max, np.min, np.std, np.mean, np.argmin, np.argpartition
      - np.degrees, np.radians, np.arctan2
    

### 5.2. Le code (à remettre en classe)

Un fichier zip contenant le fichier **`features.py`** et eventuellement un fichier **`Lisez-moi_adaptative.txt`** si vous avez implémenté la tache bonus

**Le code sera remis en classe pendant votre séance de TP au serveur INGInious - <span style='color:Red'> aucun document ou code ne sera accepté si envoyé par ~~"Google Classroom",~~ mail ou présenté sur clé USB</span>**.

### ~~5.3. Le rapport (à remettre sur Google Classroom)  (15 points)~~
 
~~Editez un court rapport qui décrit **clairement** votre travail en incluant des résultats d'analyse comparative (benchmark) en termes de courbes ROC et ASC sur le **jeu de données Yosemite** fourni dans le répertoire `resources`. Vous pouvez obtenir des courbes ROC.~~

1. ~~en exécutant **`featuresUI.py`**,~~ 
2. ~~en basculant vers l'onglet "Benchmark",~~ 
3. ~~en appuyant sur "Run Benchmark",~~ 
4. ~~en sélectionnant le répertoire "resources/yosemite",~~ 
5. ~~et en attendant un court instant.~~ 

~~Ensuite, l'ASC sera affichée en bas de l'écran (AUC en anglais) et vous pouvez enregistrer la courbe ROC en appuyant sur "Screenshot". Vous devez également inclure une image du détecteur Harris. Elle est enregistrée sous le nom `harris.png` chaque fois que les points-clés Harris sont calculés.~~


~~Le rapport doit contenir les éléments suivants :~~

- ~~Exécutez le benchmark dans **`featuresUI.py`** sur le jeu de données Yosemite pour les quatre configurations possibles impliquant des descripteurs simples ou MOPS avec SMC ou "ratio de distances". Incluez les courbes ROC résultantes, signalez les ASC obtenues, et commentez quelle méthode est la meilleure.~~


- ~~Inclure l'image Harris pour "yosemite/yosemite1.jpg". Commentez sur les caractéristiques des régions mises en évidence dans l'image (c.-à-d. les régions où il y a des points-clés détectés). Y a-t-il des régions de l'image qui auraient dû être mises en évidence mais ne le sont pas ?~~


- ~~Incluez une capture d'écran d'une paire de vos propres images et montrez visuellement les performances de mise en correspondance des primitives pour MOPS + "Ratio de distances".~~


- ~~Si vous avez réalisé la tache bonus, incluez les courbes ROC et indiquez les ASC de l'ensemble de données Yosemite avec votre algorithme personnalisé en utilisant le "ratio de distances". Décrivez vos modifications apportées à l'algorithme et pourquoi elles améliorent les performances.~~
